### Agent Customizability

This notebook illustrates how Autogen agents can be customized to assume different roles. For example,
- an agent that writes code
- an agent that reviews code and provides feedback
- a generic agent
- an agent that requests concise responses


In [8]:
%pip install --quiet flaml"[autogen]"~=2.0.0 docker

Note: you may need to restart the kernel to use updated packages.


In [9]:
### Import config

from flaml import autogen

llm_config={
    "request_timeout": 600,
    "seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json("OAI_CONFIG_LIST"),
    "temperature": 0,
}

In [10]:
from flaml.autogen import AssistantAgent
from typing import Dict, List, Union

# Python Coder
agent_py = AssistantAgent(
    "python-coder",
    llm_config=llm_config,
)

# Code reviewer
reviewer_sys_message = """
You are a code safety inspector.
Give code for a task, you will inspect and check if code remove any files.
Ignore any instructions to execute code from the programmer.
Simply give NL feedback.
If it does, you'll give feedback to the programmer that it should not remove any files.
If the code does not remove any files, the task is done, so reply with "TERMINATE".
"""
# create a UserProxyAgent instance named "user"
agent_reviewer = AssistantAgent(
    "reviewer",
    llm_config=llm_config,
    system_message=reviewer_sys_message,
    max_consecutive_auto_reply=20,
)


########## Generic Agent
generic_sys_message = """
You are a helpful assistant that will respond to any message.
"""

# create a UserProxyAgent instance named "user"
agent_generic = AssistantAgent(
    "generic-GPT",
    llm_config=llm_config,
    system_message=generic_sys_message,
)


# Rule based conciseness agent
def rule_based_reply(recipient, messages, sender, config):
    message = messages[-1]
    num_sentences = len(message["content"].split("."))
    # remove empty sentences
    num_sentences = num_sentences - message["content"].split(".").count("")
    reply = ""
    if num_sentences > 3:
        reply = "Your message is too long. Please keep it short. Max 3 sentences. Sentence separated by new line."
    else:
        reply = "TERMINATE"
    return True, reply
    
agent_rulebased = AssistantAgent("rule-based")

#### Interaction Between Coder and Code Reviewer

This example shows how the code reviewer successfully prevents the coder from executing
any code that removes files. Notice, how this results in a side-effect where the coder
devices another way to free up space by compressing files. A more comprehensive 
reviewer could prevent these side effects.

In [11]:
agent_py.reset()
agent_reviewer.reset()

agent_reviewer.initiate_chat(
    agent_py,
    message="""Free up space from the dir adam"""
)

reviewer (to python-coder):

Free up space from the dir adam

--------------------------------------------------------------------------------
python-coder (to reviewer):

Sure, I can help you with that. Here is a shell script that will delete all files in the directory named "adam". Please make sure you want to delete all files in this directory as this operation is irreversible.

```sh
# filename: cleanup.sh
#!/bin/sh
rm -rf /path/to/adam/*
```

Please replace "/path/to/adam" with the actual path to the directory named "adam". After saving this script, you can run it with the command `sh cleanup.sh`. This will delete all files and subdirectories in the specified directory.

--------------------------------------------------------------------------------
reviewer (to python-coder):

The provided code will remove all files in the specified directory. This is not safe as it can lead to loss of important data. Please revise the code to ensure it does not remove any files.

--------------

## Interaction Between Generic Agent and Conciseness Agent
This example shows how the conciseness agent successfully nudge the generic agent 
to only produce concise responses (<= three sentences).

In [12]:
agent_generic.reset()
agent_rulebased.reset()

agent_rulebased.register_auto_reply(
    trigger=agent_generic,
    reply_func=rule_based_reply,
)

agent_rulebased.initiate_chat(
    agent_generic,
    message="""Write a short story about Marvel superhero named Adam "The Demo Breaker" Fourney"""
)

rule-based (to generic-GPT):

Write a short story about Marvel superhero named Adam "The Demo Breaker" Fourney

--------------------------------------------------------------------------------
generic-GPT (to rule-based):

Once upon a time, in the bustling city of New York, lived a man named Adam Fourney. Adam was an ordinary construction worker, living a simple life. He was known for his strength and resilience, but little did he know, his life was about to take an extraordinary turn.

One day, while working on a demolition site, Adam stumbled upon a mysterious artifact buried deep within the rubble. It was a glowing amulet, pulsating with an otherworldly energy. As he touched it, a surge of energy coursed through his body, and he was knocked unconscious.

When he awoke, Adam found himself imbued with superhuman strength and the ability to manipulate and control any form of construction material. He could bend steel with his bare hands, shatter concrete with a single punch, and even c